In [1]:
import  glob
import matplotlib.pyplot as plt
from IPython.display import Audio
import zipfile
import os
import librosa
from IPython.display import Audio, display, HTML
import  sys
sys.path.append('..')
from Fuentes import audio as ta
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, LeakyReLU
from tensorflow.keras.applications import MobileNetV3Small

2025-10-20 19:56:11.135859: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-20 19:56:11.145697: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 19:56:12.156019: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 19:56:19.730359: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
DATA_DIR = '../Datos/FlowersX16/'

IMG_SIZE = (96, 96, 3)

base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=IMG_SIZE)
base_model.trainable = False

from tensorflow.keras.layers import Dropout

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),              # Añadir dropout
    Dense(128),                # Más neuronas
    LeakyReLU(alpha=0.1),
    Dropout(0.2),              # Más dropout
    Dense(64),                 # Capa adicional
    LeakyReLU(alpha=0.1),
    Dense(16, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/home/santi/Documentos/Cuarto/Deep Learning/mi_entorno/lib/python3.12/site-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(
2025-10-20 19:56:23.943662: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/santi/Documentos/Cuarto/Deep Learning/mi_entorno/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)   │ (None, 3, 3, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,022,272 (3.90 MB)

 Trainable params: 83,152 (324.81 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [3]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

SPLIT_SEED = 30
VAL_SPLIT = 0.3

dataset = tf.keras.utils.image_dataset_from_directory(
    directory=DATA_DIR,
    image_size=IMG_SIZE[:2],
    batch_size=32,
    label_mode='categorical',
    validation_split=VAL_SPLIT,
    subset='training',
    shuffle=True,
    seed=SPLIT_SEED,
)

dataset_val = tf.keras.utils.image_dataset_from_directory(
    directory=DATA_DIR,
    image_size=IMG_SIZE[:2],
    batch_size=32,
    label_mode='categorical',
    validation_split=VAL_SPLIT,
    subset='validation',
    shuffle=True,
    seed=SPLIT_SEED,
)

# Añadir augmentación de datos
dataset = dataset.map(lambda x, y: (tf.keras.applications.mobilenet_v3.preprocess_input(x), y))
dataset_val = dataset_val.map(lambda x, y: (tf.keras.applications.mobilenet_v3.preprocess_input(x), y))

epocas = 100
paciencia = 15

early_stop = EarlyStopping(monitor='val_loss', patience=paciencia, restore_best_weights=True, min_delta=0.001)

H = model.fit(
    dataset,
    epochs=epocas,
    validation_data=dataset_val,
    callbacks=[early_stop],
)


Found 15740 files belonging to 16 classes.
Using 11018 files for training.
Found 15740 files belonging to 16 classes.
Using 4722 files for validation.
Epoch 1/100
345/345 ━━━━━━━━━━━━━━━━━━━━ 68s 162ms/step - accuracy: 0.5156 - loss: 1.5029 - val_accuracy: 0.7103 - val_loss: 0.8917
Epoch 2/100
345/345 ━━━━━━━━━━━━━━━━━━━━ 49s 143ms/step - accuracy: 0.6648 - loss: 1.0172 - val_accuracy: 0.7463 - val_loss: 0.7875
Epoch 3/100
345/345 ━━━━━━━━━━━━━━━━━━━━ 35s 103ms/step - accuracy: 0.7021 - loss: 0.9045 - val_accuracy: 0.7490 - val_loss: 0.7585
Epoch 4/100
345/345 ━━━━━━━━━━━━━━━━━━━━ 36s 104ms/step - accuracy: 0.7278 - loss: 0.8337 - val_accuracy: 0.7626 - val_loss: 0.7289
Epoch 5/100
345/345 ━━━━━━━━━━━━━━━━━━━━ 36s 104ms/step - accuracy: 0.7365 - loss: 0.7934 - val_accuracy: 0.7702 - val_loss: 0.7140
Epoch 6/100
345/345 ━━━━━━━━━━━━━━━━━━━━ 35s 103ms/step - accuracy: 0.7437 - loss: 0.7700 - val_accuracy: 0.7626 - val_loss: 0.7208
Epoch 7/100
345/345 ━━━━━━━━━━━━━━━━━━━━ 37s 107ms/step -

In [4]:
model.evaluate(dataset_val)

148/148 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - accuracy: 0.7829 - loss: 0.6800


[0.6800406575202942, 0.7829309701919556]